In [ ]:
import random
import math

In [ ]:
class EllipticCurve:
    def __init__(self, a, b, n):
        self.a = a
        self.b = b
        self.n = n

class Point:
    def __init__(self, X, Y, Z):
        self.X = X
        self.Y = Y
        self.Z = Z
    
    def set(self, point):
        self.X, self.Y, self.Z = point.getPoint()

    def setPoint(self, X, Y, Z):
        self.X = X
        self.Y = Y
        self.Z = Z
    
    def getPoint(self):
        return self.X, self.Y, self.Z

    def xy2xyz(self):
        self.Z = 1
    
    def xyz2xy(self):
        self.X /= self.Z
        self.Y /= self.Z

    def doubling(self, curve):
        X, Y, Z = self.getPoint()
        a = curve.a
        n = curve.n

        if Y == 0:
            self.setPoint(0, 1, 0)
            return

        if Z == 0: return
        
        A = a * pow(Z, 2, n) + 3 * pow(X, 2, n)
        B = Y * Z
        C = X * Y * B
        H = pow(A, 2, n) - 8 * C
        X2 = (2 * H * B) % n
        Y2 = (A * (4 * C - H) - 8 * pow(Y, 2, n) * pow(B, 2, n)) % n
        Z2 = pow(2 * B, 3, n)
        self.setPoint(X2, Y2, Z2)
    
    def add(self, point, curve):
        X1, Y1, Z1 = self.getPoint()
        X2, Y2, Z2 = point.getPoint()

        if Z1 == 0: return self.setPoint(X2, Y2, Z2)
        if Z2 == 0: return
        
        a, b, n = curve.a, curve.b, curve.n
        
        A1 = Y2 * Z1 % n
        A2 = Y1 * Z2 % n
        B1 = X2 * Z1 % n
        B2 = X1 * Z2 % n

        if B1 == B2:
            if A1 == A2:
                self.doubling(curve)
            else:
                self.setPoint(0, 1, 0)
            return

        B = (B1 - B2) % n
        A = (A1 - A2) % n
        E = (Z1 * Z2) % n
        C = pow(A, 2, n) * E - pow(B, 3, n) - 2 * pow(B, 2, n) * B2
        X3 = (B * C) % n
        Y3 = (A * (pow(B, 2, n) * B2 - C) - pow(B, 3, n) * A2) % n
        Z3 = (pow(B, 3, n) * E) % n
        self.setPoint(X3, Y3, Z3)
    
    def kPmultiply(self, k, curve):
        if k == 1:
            return
        P2 = Point(0, 1, 0)
        k2 = 0
        bit = 1 << (len(bin(k)) - 3)
        
        while k != k2:
            # 2P
            k2 = k2 << 1
            if k2: 
                P2.doubling(curve)
            # P + Q
            if k & bit:
                k2 += 1
                P2.add(self, curve)
            bit = bit >> 1
        self.set(P2)

In [ ]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def RandomCurve(n):
    return random.randint(0, n), random.randint(0, n), random.randint(0, n)
  
def MiillerRabin(d, n): 
    a = 2 + random.randint(1, n - 4); 
  
    x = pow(a, d, n)
  
    if (x == 1 or x == n - 1): 
        return True; 

    while (d != n - 1): 
        x = (x * x) % n; 
        d *= 2; 
  
        if (x == 1): 
            return False; 
        if (x == n - 1): 
            return True; 
  
    return False;

def isPrime(n): 
    if (n <= 1 or n == 4): 
        return False; 
    if (n <= 3): 
        return True; 
  
    t = n - 1; 
    while (t % 2 == 0): 
        t //= 2; 

    if (MiillerRabin(t, n) == False): 
        return False; 
  
    return True;

In [ ]:
def Lenstre(N, B, curves):
    start_time = time.time()
    for i in range(curves):
        g = N
        if g == N:
            x, y, a = RandomCurve(N)
            b = (pow(y, 2, N) - pow(x, 3, N) - a * x) % N
            g = gcd(N, 4 * pow(a, 3) + 27 * pow(b, 2))
        
        if 1 < g < N:
            return {'Factorization': g, 'Curves': i + 1, 'X': x, 'Y': y, 'A': a, 'time': time.time() - start_time}

        P = Point(x, y, 1)
        curve = EllipticCurve(a, b, N)

        M = 1
        for p in range(2, B):
            if isPrime(p):
                M *= pow(p, int(math.log(B, p)))

        P.kPmultiply(M, curve)
        gcd_nz = gcd(N, P.Z)
        if gcd_nz > 1:
            return {'Factorization': gcd_nz, 'Curves': i + 1, 'X': x, 'Y': y, 'A': a, 'time': time.time() - start_time}
    return None

In [ ]:
N = 864248863691854142077648660979
B = 700
curveN = 100000000

In [ ]:
Lenstre(N, B, curveN)

{'A': 26596247360763099585176515070,
 'Curves': 9,
 'Factorization': 919260053237879,
 'X': 823024219972408971141486829477,
 'Y': 45062646292285755280721957492,
 'time': 0.631033182144165}